# AYA Airport

### Import

In [ ]:
from random import choice, randint
import pandas as pd
from time import sleep
import datetime
import matplotlib.pyplot as plt
from skimage.io import imread, imshow
from google.colab import output

### Global variables

In [ ]:
global dict_floor_queues #waiting queues to enter the runways
dict_floor_queues = {'queue_1':{}, 'queue_2':{}, 'queue_3':{}}

global dict_sky_queues #waiting queues to enter the runways
dict_sky_queues = {'queue_1':{}, 'queue_2':{}}

global dict_runways #airplane runways
dict_runways = {'runway_1':{}, 'runway_2':{}, 'runway_3':{}}

global dict_available_spaces
dict_available_spaces = {0:3,1:2,2:1,3:0} #parameterizing queues availability 

global dict_runway_to_queue #relating queues to runways
dict_runway_to_queue = {'runway_1':'queue_1','runway_2':'queue_2','runway_3':'queue_3'}

global dict_queue_to_runway #relating runways to queues
dict_queue_to_runway = {'queue_1':'runway_1','queue_2':'runway_2','queue_3':'runway_3'}


global global_dict_crashed_airplanes
global_dict_crashed_airplanes = {}
global_dict_crashed_airplanes['CURRENT TIME'] = {}

global airplane_crash #number of airplanes crashing
airplane_crash = 0

global emergency_landings #number of emergency landings
emergency_landings = 0


### Functions

#### Function ***create_airplanes_floor***

In [ ]:
def create_airplanes_floor(dict_floor_queues, dict_available_spaces): #creating airplanes to floor queues
  '''
  if the airplane is already on queue, and new ones are added, it means that in a round it doesn't move.
  So there's added one more unit of time to the airplane. 
  '''
  for queue in dict_floor_queues:
    for airplane in dict_floor_queues[queue]:
      dict_floor_queues[queue][airplane]['Time To Take Off'] += 1

  #Queues must be consulted
  for queue in dict_floor_queues:
    airplanes_on_queue = len(dict_floor_queues[queue])
    available_queues_spaces = dict_available_spaces[airplanes_on_queue]

    #TO ID
    letters = 'ABCDEFGIJKLMNOPQSTUVWXYZ'
    numbers = '123456789'

    #the random number of airplanes must be between zero and the maximum of spaces available on the queues
    number_of_airplanes = randint(0,available_queues_spaces) 
    for i in range(number_of_airplanes):
      ID = [choice(letters) for x in range(3)] + [choice(numbers) for x in range(3)] 
      ID = ''.join(ID)

      while (ID in dict_floor_queues[queue]):
        ID = [choice(letters) for x in range(3)] + [choice(numbers) for x in range(3)] 
        ID = ''.join(ID)

      time_to_take_off = 0
      airplane_fuel = randint(2,20)
      dict_airplane = {'Time To Take Off':time_to_take_off, 'Fuel':airplane_fuel}
      dict_floor_queues[queue][ID] = dict_airplane

  return dict_floor_queues

#### Function ***enter_runway***

In [ ]:
def enter_runway(dict_runways, dict_floor_queues, dict_runway_to_queue, dict_available_spaces):
  '''
  if the airplane is already on runway, and new ones are added, it means that in a round the airplane doesn't move.
  So there's added one more unit of time to the airplane. 
  '''
  #mean_time = []
  for runway in dict_runways:
    for airplane in dict_runways[runway]:
      dict_runways[runway][airplane]['Time To Take Off'] += 1
      #mean_time.append( dict_runways[runway][airplane]['Time To Take Off'])

  #Runways must be consulted
  list_airplanes_out_of_runway = []
  for runway in dict_runways:
    list_airplanes_in_queues = list(dict_floor_queues[dict_runway_to_queue[runway]].keys())
    airplanes_on_runway = len(dict_runways[runway])
    available_runways_spaces = dict_available_spaces[airplanes_on_runway]


    for ID in dict_floor_queues[dict_runway_to_queue[runway]]:
      #Adding new airplanes while there's still spaces available on runways
        if (available_runways_spaces > 0):
          airplane_to_runway = list_airplanes_in_queues[0]
          list_airplanes_out_of_runway.append(airplane_to_runway)
          dict_runways[runway][ID]= dict_floor_queues[dict_runway_to_queue[runway]][ID]
          list_airplanes_in_queues.remove(airplane_to_runway)
          available_runways_spaces -= 1
  
  '''
  Once the airplane is added to the runways, it must be removed from the floor queues.
  '''
  for runway in dict_runways:
    for airplane in list_airplanes_out_of_runway:
      if (airplane in dict_floor_queues[dict_runway_to_queue[runway]]):
        dict_floor_queues[dict_runway_to_queue[runway]].pop(airplane)

  return dict_runways

#### Function ***takeoff***

In [ ]:
def takeoff(dict_runways, dict_runway_to_queue):
    '''
    for each round, the first airplane to enter in respective runway is gonna be the one that flies.
    '''
    dict_airplanes_to_take_off = {'runway_1':{}, 'runway_2':{}, 'runway_3':{}} 
    list_airplanes_take_off = []

    #Checking runways
    for runway in dict_runways:
        list_airplanes_in_runway = list(dict_runways[runway].keys())
        if (len(list_airplanes_in_runway)!=0):
            airplane_to_take_off =  list_airplanes_in_runway[0]
            list_airplanes_take_off.append(airplane_to_take_off)
            dict_airplanes_to_take_off[runway][airplane_to_take_off] = dict_runways[runway][airplane_to_take_off]

    
    #Once the airplane took off, it must be removed from the runway.
    for runway in dict_runways:
      for airplane in list_airplanes_take_off:
        if (airplane in dict_runways[runway]):
          dict_runways[runway].pop(airplane)
    return dict_airplanes_to_take_off

#### Function ***create_airplanes_sky***

In [ ]:
def create_airplanes_sky(dict_sky_queues, dict_available_spaces):
    '''
    Once the airplane is already on the queue and new airplanes are added, it means
    that in a round the airplane doesn't move 
    '''
    for queue in dict_sky_queues:
      for airplane in dict_sky_queues[queue]:
        dict_sky_queues[queue][airplane]['Time To Land'] += 1
        dict_sky_queues[queue][airplane]['Fuel'] -= 1
  
  #Queues must be consulted
    for queue in dict_sky_queues:
        airplanes_on_queue = len(dict_sky_queues[queue])
        available_queues_spaces = dict_available_spaces[airplanes_on_queue]

        #TO ID
        letters = 'ABCDEFGIJKLMNOPQSTUVWXYZ'
        numbers = '123456789'
        
        number_of_airplanes = randint(0,available_queues_spaces)
        for i in range(number_of_airplanes): 
          ID = [choice(letters) for x in range(3)] + [choice(numbers) for x in range(3)] 
          ID = ''.join(ID)

          while (ID in dict_sky_queues[queue]):
              ID = [choice(letters) for x in range(3)] + [choice(numbers) for x in range(3)] 
              ID = ''.join(ID)

          time_to_land = 0
          airplane_fuel = randint(2,20)
          dict_airplane = {'Time To Land':time_to_land, 'Fuel':airplane_fuel}
          dict_sky_queues[queue][ID] = dict_airplane
    return dict_sky_queues

#### Function ***landing***

In [ ]:
def landing(dict_runways, dict_sky_queues, dict_runway_to_queue, dict_queue_to_runway, emergency_landings, airplane_crash):
    dict_airplanes_normal_landings = {'runway_1':{}, 'runway_2':{}, 'runway_3':{}}
    dict_airplanes_on_emergency_landings = {'runway_1':{}, 'runway_2':{}, 'runway_3':{}}
    dict_crashed_airplanes = {'queue_1':{}, 'queue_2':{}, 'queue_3':{}}
    list_queues = [queue for queue in dict_sky_queues]
    for queue in dict_sky_queues:
        for airplane in dict_sky_queues[queue]:
          if(dict_sky_queues[queue][airplane]['Fuel']==1):
            verify_airplane = True
            i = 0
            while verify_airplane == True:
                if (i<2):
                  if (len(dict_airplanes_on_emergency_landings[dict_queue_to_runway[list_queues[i]]].keys())==0):
                    dict_airplanes_on_emergency_landings[dict_queue_to_runway[list_queues[i]]][airplane] = dict_sky_queues[queue][airplane]
                    emergency_landings += 1
                    verify_airplane = False
                  else:
                    i += 1
                elif (i==2):
                  if (len(dict_airplanes_on_emergency_landings['runway_3'].keys())==0):
                    dict_airplanes_on_emergency_landings['runway_3'][airplane] = dict_sky_queues[queue][airplane]
                    verify_airplane = False
                  else:
                    airplane_crash += 1
                    dict_crashed_airplanes[queue][airplane] = dict_sky_queues[queue][airplane]
                    verify_airplane = False
              
          else:
            if (len(dict_airplanes_normal_landings[dict_queue_to_runway[queue]].keys())==0):
              dict_airplanes_normal_landings[dict_queue_to_runway[queue]][airplane] = dict_sky_queues[queue][airplane]

    #removing crashed airplanes from sky queues 
    for queue in dict_crashed_airplanes:
      for airplane in dict_crashed_airplanes[queue]:
          if (airplane in dict_sky_queues[queue]):
            dict_sky_queues[queue].pop(airplane)

    #removing airplanes on emergency landings from sky queues 
    for runway in dict_airplanes_on_emergency_landings:
      for airplane in dict_airplanes_on_emergency_landings[runway]:
        for queue in dict_sky_queues:
          if (airplane in dict_sky_queues[queue]):
            dict_sky_queues[queue].pop(airplane)

    #removing airplanes on normal landings from sky queues 
    for runway in dict_airplanes_normal_landings:
      for airplane in dict_airplanes_normal_landings[runway]:
        for queue in dict_sky_queues:
          if (airplane in dict_sky_queues[queue]):
            dict_sky_queues[queue].pop(airplane)
        
    return dict_crashed_airplanes, dict_airplanes_on_emergency_landings, dict_airplanes_normal_landings

#### Function ***format_line_name***

In [ ]:
def format_line_name(line_name:str):
  name = line_name.split('_')[0].title()
  number = line_name.split('_')[1]

  output = [name, ' ',number]

  return ''.join(output)

#### Function ***df_to_print***

In [ ]:
def df_to_print(dictionary: dict,round:int):
  rounds = []
  lines = []
  airplanes = []
  times = []
  fuels = []
  for line in dictionary:
    for airplane in dictionary[line]:
      rounds.append(round)
      line_name = format_line_name(line)
      lines.append(line_name)
      airplanes.append(airplane)
      fuel = dictionary[line][airplane]['Fuel']
      fuels.append(fuel)
      if ('runway_1' in dictionary):
        if ('Time To Take Off' in dictionary[line][airplane].keys()):
          time = dictionary[line][airplane]['Time To Take Off']
          times.append(time)
        else:
           time = dictionary[line][airplane]['Time To Land']
           times.append(time) 
      else: 
        if ('Time To Take Off' in dictionary[line][airplane].keys()):
          time = dictionary[line][airplane]['Time To Take Off']
          times.append(time)
        else:
           time = dictionary[line][airplane]['Time To Land']
           times.append(time)
  if (len(times) != 0):
    mean_time = sum(times)/len(times)
  else:
    mean_time = 'No airplanes.'
    
  df = pd.DataFrame()
  df= df.assign(ROUND=rounds).assign(AIRPLANE=airplanes).assign(LINE=lines).assign(TIME=times).assign(FUEL=fuels)

  return df, mean_time

#### Function ***output_generator***

In [ ]:

def output_generator(final_output_json, key, dict_name, dictionary, all_df, round_of_execution):
  #exhibition of the dataframe 
  df, mean_time = df_to_print(dictionary, round_of_execution)
  N_SPACES = 5
  #head = '\n'+'--'*N_SPACES + ' '+ dict_name + ' '+'--'*N_SPACES
  print('\n')
  # print(head)
  # print('\n')
  if (len(df.index)!=0):
    print(df)
    print(f'\nMEAN TIME: {round(mean_time,2)} unities of time')
  else:
    print('')
  all_df.append(pd.DataFrame([{'AIRPLANE': dict_name}]))
  all_df.append(df)
  final_output_json[key][dict_name] = dictionary


#### Function ***open_images***

In [ ]:
def open_image(dict_name, dictionary, list_imgs, show_image):
  if (show_image == True):
    name_to_display = {'FLOOR QUEUES':' FLOOR QUEUE', 'RUNWAYS': 'RUNWAY', 'TAKING OFF':'TAKING OFF', 'SKY QUEUES':'SKY QUEUE',\
                      'CRASHED AIRPLANES':'CRASHED AIRPLANE', 'EMERGENCY LANDINGS':'EMERGENCY LANDING', 'LANDINGS':'LANDING'}
    print('\n')
    print('**'*5 + ' ' +dict_name.upper()+ ' '+ '**'*5)
    print('\n')
    spaces = 10-len(name_to_display[dict_name])
    # head = ' '*2 + f'{name_to_display[dict_name]} 1' + ' '*spaces + f'{name_to_display[dict_name]} 2' + ' '*spaces + f'{name_to_display[dict_name]} 3'
    # print(head.center(25))
    #print('\n')
    plt.figure(figsize=(4.5, 4.5))
    i = 0
    for line in dictionary: 
      number_of_airplanes = len(dictionary[line])
      img = list_imgs[number_of_airplanes]
      plt.subplot(3, 3, i+1)
      plt.imshow(img)
      plt.axis('off')
      i += 1

      plt.tight_layout()
    return plt.show()

#### Function ***save_imgs***

##### The images are available [here](https://drive.google.com/drive/folders/1-yCwRfoWWo9Cpwg6Aev2A4uNjaU09cd7?usp=sharing)



In [ ]:
def save_imgs(list_imgs, number_of_imgs, names_imgs):
  for i in range(number_of_imgs):
    img = imread(f'{names_imgs}_{i}.png')
    list_imgs.append(img)

In [ ]:
runways_imgs =[]
queues_imgs =[]
taking_off_imgs = []
crashed_airplanes_imgs = []
emergency_landings_imgs = []
landings_imgs = []

save_imgs(runways_imgs, 4, 'runway')
save_imgs(queues_imgs, 4, 'queue')
save_imgs(taking_off_imgs, 2, 'taking_off')
save_imgs(crashed_airplanes_imgs, 2, 'crashed_airplane')
save_imgs(emergency_landings_imgs, 2, 'emergency_landing')
save_imgs(landings_imgs, 2, 'landing')

### Main

In [ ]:
print('Welcome to AYA Airport!\n')

all_dicts_names = ['FLOOR QUEUES', 'RUNWAYS','TAKING OFF', 'SKY QUEUES','CRASHED AIRPLANES', 'EMERGENCY LANDINGS', 'LANDINGS']
moment_of_execution = str(datetime.datetime.now())
final_output_json = {'CURRENT TIME': {}}
all_df = []
show_image = True
save_outputs = False
SLEEP_TIME = 3
NUMBER_OF_ROUNDS = 10


for round_of_execution in range(NUMBER_OF_ROUNDS):
  N_SPACES = 5

  # print('\n')
  # print('**'*N_SPACES + f' ROUND {round_of_execution} '+'**'*N_SPACES)
  # print('\n')

  final_output_json[f'ROUND {round_of_execution}'] = {}
  dict_floor_queues = create_airplanes_floor(dict_floor_queues=dict_floor_queues, dict_available_spaces=dict_available_spaces)
  open_image(dict_name=all_dicts_names[0], dictionary=dict_floor_queues, list_imgs=queues_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key='CURRENT TIME', dict_name=all_dicts_names[0], dictionary=dict_floor_queues,\
                   all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear()

  dict_runways = enter_runway(dict_runways=dict_runways, dict_floor_queues=dict_floor_queues,\
                              dict_runway_to_queue=dict_runway_to_queue, dict_available_spaces=dict_available_spaces)
  open_image(dict_name=all_dicts_names[1], dictionary=dict_runways, list_imgs=runways_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key='CURRENT TIME', dict_name=all_dicts_names[1], dictionary=dict_runways,\
                   all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear()
  

  dict_airplanes_to_take_off = takeoff(dict_runways=dict_runways, dict_runway_to_queue=dict_runway_to_queue)
  open_image(dict_name=all_dicts_names[2], dictionary=dict_airplanes_to_take_off, list_imgs=taking_off_imgs,show_image=show_image)
  output_generator(final_output_json=final_output_json, key=f'ROUND {round_of_execution}', dict_name=all_dicts_names[2], \
                   dictionary=dict_airplanes_to_take_off, all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear()

  dict_sky_queues = create_airplanes_sky(dict_sky_queues=dict_sky_queues, dict_available_spaces=dict_available_spaces)
  open_image(dict_name=all_dicts_names[3], dictionary=dict_sky_queues, list_imgs=queues_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key='CURRENT TIME', dict_name=all_dicts_names[3], dictionary=dict_sky_queues, \
                   all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear()

  dict_crashed_airplanes, dict_airplanes_emergency_landings, dict_airplanes_normal_landings =landing(dict_sky_queues=dict_sky_queues, dict_runways=dict_runways,\
                                 dict_runway_to_queue=dict_runway_to_queue, dict_queue_to_runway=dict_queue_to_runway,\
                                 emergency_landings=emergency_landings, airplane_crash=airplane_crash)
  
  final_output_json['CURRENT TIME']['CRASHED AIRPLANES'] = dict_crashed_airplanes

  open_image(dict_name=all_dicts_names[4], dictionary=dict_crashed_airplanes, list_imgs=crashed_airplanes_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key=f'ROUND {round_of_execution}', dict_name=all_dicts_names[4], \
                   dictionary=dict_crashed_airplanes, all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear() 
   

  open_image(dict_name=all_dicts_names[5], dictionary=dict_airplanes_emergency_landings, list_imgs=emergency_landings_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key=f'ROUND {round_of_execution}', dict_name=all_dicts_names[5], \
                   dictionary=dict_airplanes_emergency_landings, all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear() 
  
  open_image(dict_name=all_dicts_names[6], dictionary=dict_airplanes_normal_landings, list_imgs=landings_imgs, show_image=show_image)
  output_generator(final_output_json=final_output_json, key=f'ROUND {round_of_execution}', dict_name=all_dicts_names[6], \
                   dictionary=dict_airplanes_normal_landings, all_df=all_df, round_of_execution=round_of_execution)
  sleep(SLEEP_TIME)
  output.clear() 
  
  #sleep(5)

if (save_outputs==True):
  #saving DataFrame on .xlsx
  final_df = pd.concat(all_df)
  final_df = final_df.reset_index()
  final_df.drop(["index"], axis=1, inplace=True)
  exec_moment = moment_of_execution.replace(':','-').replace(' ','-')
  final_df.to_excel(f'AYA_Airport_Output_{exec_moment}.xlsx')

  #saving the data on .json
  with open(f'AYA_Airport_Output_{exec_moment}.json','a') as file:
    file.write(str(final_output_json))

  #saving the data on .json
  with open(f'AYA_Airport_Output_{exec_moment}.txt','a') as file:
    file.write(f'Number of rounds: {NUMBER_OF_ROUNDS}')
    file.write(f'\nTotal of airplane crashed: {airplane_crash}')